In [ ]:
import numpy as np
import scipy.linalg as la

##############################################################################
# Hilfsfunktionen für Persönlichkeitszustände und Ausgabe
##############################################################################

def normalize_state(psi):
    """Normiert einen Zustandsvektor."""
    return psi / np.linalg.norm(psi)

def print_personality_2(name, state1, state2, label1="Merkmal 1", label2="Merkmal 2"):
    p_strong1 = np.abs(state1[1])**2
    p_weak1   = np.abs(state1[0])**2
    p_strong2 = np.abs(state2[1])**2
    p_weak2   = np.abs(state2[0])**2
    print(f"{name}:")
    print(f"  {label1}: stark = {p_strong1:.3f}, gering = {p_weak1:.3f}")
    print(f"  {label2}: stark = {p_strong2:.3f}, gering = {p_weak2:.3f}")
    print()

def print_personality_5(name, states, labels):
    print(f"{name}:")
    for label, st in zip(labels, states):
        p_strong = np.abs(st[1])**2
        p_weak   = np.abs(st[0])**2
        print(f"  {label}: stark = {p_strong:.3f}, gering = {p_weak:.3f}")
    print()

def kron_all(list_of_vectors):
    out = list_of_vectors[0]
    for v in list_of_vectors[1:]:
        out = np.kron(out, v)
    return out

##############################################################################
# XX-Kopplung, blockweise
##############################################################################

def sigma_x():
    return np.array([[0,1],[1,0]], dtype=complex)

def make_xx_gate_4x4(gamma):
    from scipy.linalg import expm
    xx = np.kron(sigma_x(), sigma_x())
    ham_4x4 = -1j * gamma * xx
    return expm(ham_4x4)

def apply_two_qubit_gate(psi_vec, gate_4x4, qubit1, qubit2, N):
    if qubit2 < qubit1:
        qubit1, qubit2 = qubit2, qubit1
    shape_psi = [2] * N
    psi_resh = np.reshape(psi_vec, shape_psi)
    axes_order = list(range(N))
    axes_order.remove(qubit1)
    axes_order.remove(qubit2)
    axes_order.append(qubit1)
    axes_order.append(qubit2)
    psi_perm = np.transpose(psi_resh, axes_order)
    dim_rest = 2 ** (N - 2)
    psi_perm = np.reshape(psi_perm, (dim_rest, 4))
    new_psi_perm = psi_perm @ gate_4x4.T
    new_psi_perm = np.reshape(new_psi_perm, (dim_rest, 2, 2))
    inverse_order = [0] * N
    for k, val in enumerate(axes_order):
        inverse_order[val] = k
    new_psi = np.reshape(new_psi_perm, [2]*N)
    new_psi = np.transpose(new_psi, np.argsort(inverse_order))
    return np.reshape(new_psi, (2 ** N,))

##############################################################################
# Dissipative Kraus-Operatoren: Amplitude-Damping-Kanal
##############################################################################

def amplitude_damping_for_qubit(q, gamma, N):
    full_dim = 2**N
    K0 = np.zeros((full_dim, full_dim), dtype=complex)
    K1 = np.zeros((full_dim, full_dim), dtype=complex)
    for i in range(full_dim):
        if ((i >> q) & 1) == 0:
            K0[i,i] = 1.0
        else:
            K0[i,i] = np.sqrt(1 - gamma)
            j = i & ~(1 << q)
            K1[j, i] = np.sqrt(gamma)
    return [K0, K1]

def apply_dissipation_step(psi_vec_or_rho, qubit_list, gamma, N):
    if psi_vec_or_rho.ndim == 1:
        rho = np.outer(psi_vec_or_rho, psi_vec_or_rho.conjugate())
    elif psi_vec_or_rho.ndim == 2:
        rho = psi_vec_or_rho
    else:
        raise ValueError("Input must be 1D or 2D.")
    for q in qubit_list:
        Ks = amplitude_damping_for_qubit(q, gamma, N)
        new_rho = np.zeros_like(rho, dtype=complex)
        for K in Ks:
            new_rho += K @ rho @ K.conjugate().T
        rho = new_rho
    return rho

##############################################################################
# Entropie-Berechnungen (global & Subsystem) IN BITS
##############################################################################

def compute_vn_entropy(rho):
    """
    S(rho) = -Tr(rho log2 rho).
    => Einheit: bits
    """
    eigvals = la.eigvalsh(rho)
    eigvals = np.clip(eigvals, 1e-15, None)
    return -np.sum(eigvals * np.log2(eigvals))

def partial_trace(rho, qubits_to_keep, N):
    qubits_to_keep = sorted(qubits_to_keep)
    if len(qubits_to_keep) == N:
        return rho

    rho_reshaped = np.reshape(rho, [2]*2*N)
    all_qubits = list(range(N))
    remove_qubits = [q for q in all_qubits if q not in qubits_to_keep]

    keep_pos = [all_qubits.index(k) for k in qubits_to_keep]
    remove_pos = [all_qubits.index(r) for r in remove_qubits]

    new_order = (
        keep_pos + remove_pos +
        [p+N for p in keep_pos] + [p+N for p in remove_pos]
    )

    rho_trans = np.transpose(rho_reshaped, new_order)

    dim_keep = 2**len(qubits_to_keep)
    dim_remove = 2**(N - len(qubits_to_keep))
    rho_trans = np.reshape(rho_trans, (dim_keep, dim_remove, dim_keep, dim_remove))

    rho_reduced = np.zeros((dim_keep, dim_keep), dtype=complex)
    for i in range(dim_remove):
        rho_reduced += rho_trans[:, i, :, i]
    return rho_reduced

def compute_phi(rho, partitions, N):
    """
    Phi = sum(S(rho_part)) - S(rho).
    Entropie = base2 => bits
    """
    S_full = compute_vn_entropy(rho)
    S_sum = 0.0
    for part in partitions:
        rho_part = partial_trace(rho, part, N)
        S_sum += compute_vn_entropy(rho_part)
    return S_sum - S_full

##############################################################################
# "Fine" und "Coarse" Entropie-Auswertung (global) in BITS
##############################################################################

def print_fine_grained(psi_or_rho, N, threshold=0.1):
    if psi_or_rho.ndim == 1:
        p_out = np.abs(psi_or_rho)**2
    else:
        p_out = np.real(np.diag(psi_or_rho))
    nonzero = p_out[p_out > 0]
    # Shannon Entropie in base2 => bits
    S_obs = -np.sum(nonzero * np.log2(nonzero))
    print(f"Fine-Grained Observational Entropy = {S_obs:.6f} bits\n")
    print("Fein-granulare Messresultate")
    if threshold is not None:
        print(f"(nur p > {threshold})")
    else:
        print("(alle Bitstrings)")
    for outcome in range(2**N):
        prob = p_out[outcome]
        if threshold is not None and prob <= threshold:
            continue
        bitstring = format(outcome, f'0{N}b')
        print(f"{bitstring}: {prob:.6f}")
    return S_obs

def fine_to_coarse(bitstring, n_traits=2):
    coarse = ""
    for person_idx in range(3):
        start = person_idx * n_traits
        end = start + n_traits
        segment = bitstring[start:end]
        if all(c == '1' for c in segment):
            coarse += "1"
        else:
            coarse += "0"
    return coarse

def coarse_volume(c_label, n_traits=2):
    vol = 1
    if n_traits == 2:
        for c in c_label:
            vol *= 1 if c == '1' else 3
    else:
        for c in c_label:
            vol *= 1 if c == '1' else 31
    return vol

def print_coarse_grained(psi_or_rho, N, n_traits=2, threshold=0.1):
    if psi_or_rho.ndim == 1:
        p_out = np.abs(psi_or_rho)**2
    else:
        p_out = np.real(np.diag(psi_or_rho))
    coarse_probs = {}
    for outcome in range(2**N):
        prob = p_out[outcome]
        bitstring = format(outcome, f'0{N}b')
        c = fine_to_coarse(bitstring, n_traits)
        coarse_probs[c] = coarse_probs.get(c, 0) + prob
    S_coarse = 0
    for c_label, pr in coarse_probs.items():
        if pr > 0:
            vol = coarse_volume(c_label, n_traits)
            S_coarse -= pr * np.log2(pr / vol)
    print(f"Coarse-Grained Observational Entropy = {S_coarse:.6f} bits\n")
    print("Coarse-Grained Messresultate")
    if threshold is not None:
        print(f"(nur p > {threshold})")
    else:
        print("(alle)")
    for c_label, pr in coarse_probs.items():
        if threshold is not None and pr <= threshold:
            continue
        vol = coarse_volume(c_label, n_traits)
        print(f"{c_label} (Volumen={vol}): p = {pr:.6f}")
    return S_coarse

##############################################################################
# Subsystem Fine/Coarse + VNE in bits
##############################################################################

def print_fine_grained_subsystem(rho_sub, n_traits, threshold=0.01):
    p_out = np.real(np.diag(rho_sub))
    nonzero = p_out[p_out > 0]
    S_obs = -np.sum(nonzero * np.log2(nonzero))  # bits
    print(f"  Fine-Entropy (Subsystem) = {S_obs:.6f} bits")
    print(f"  (nur p > {threshold})")
    dim = 2**n_traits
    for outcome in range(dim):
        prob = p_out[outcome]
        if prob <= threshold:
            continue
        bitstring = format(outcome, f'0{n_traits}b')
        print(f"    {bitstring}: {prob:.6f}")
    return S_obs

def fine_to_coarse_subsystem(bitstring):
    return '1' if all(c == '1' for c in bitstring) else '0'

def coarse_volume_subsystem(label, n_traits):
    if label == '1':
        return 1
    else:
        return 3**n_traits

def print_coarse_grained_subsystem(rho_sub, n_traits=2, threshold=0.01):
    p_out = np.real(np.diag(rho_sub))
    coarse_probs = {}
    dim = 2**n_traits
    for outcome in range(dim):
        prob = p_out[outcome]
        bitstring = format(outcome, f'0{n_traits}b')
        c = fine_to_coarse_subsystem(bitstring)
        coarse_probs[c] = coarse_probs.get(c, 0) + prob

    S_coarse = 0
    for c_label, pr in coarse_probs.items():
        if pr > 0:
            vol = coarse_volume_subsystem(c_label, n_traits)
            S_coarse -= pr * np.log2(pr / vol)

    print(f"  Coarse-Entropy (Subsystem) = {S_coarse:.6f} bits")
    print(f"  (nur p > {threshold})")
    for c_label, pr in coarse_probs.items():
        if pr <= threshold:
            continue
        vol = coarse_volume_subsystem(c_label, n_traits)
        print(f"    {c_label} (Vol={vol}): p={pr:.6f}")
    return S_coarse

def print_subsystem_entropies(rho, sub_qubits, n_traits, label, threshold=0.01):
    N = 3 * n_traits
    rho_sub = partial_trace(rho, sub_qubits, N)
    vn_sub = compute_vn_entropy(rho_sub)  # bits
    print(f"\n--- Subsystem {label} (Qubits={sub_qubits}) ---")
    print(f"  Von-Neumann Entropy (Subsystem) = {vn_sub:.6f} bits")
    _ = print_fine_grained_subsystem(rho_sub, n_traits, threshold=threshold)
    _ = print_coarse_grained_subsystem(rho_sub, n_traits, threshold=threshold)

##############################################################################
# HAUPTFUNKTION (Beispiel)
##############################################################################

def run_program_with_handcrafted_traits(n_traits=2, threshold=0.01, p_dissip=0.5, num_diss_steps=5):
    """
    3 Personen, n_traits Qubits pro Person => N=3*n_traits
    -> Fine/Coarse Entropien in bits, von-Neumann Entropien in bits
    -> phi( A|B|C ) in bits
    """
    assert n_traits in [2,5]
    N = 3 * n_traits

    print(f"==== TEAM-SYSTEM mit {n_traits} Traits/Person, p_dissip={p_dissip}, {num_diss_steps} Dissipationsschritte ====\n")

    partitions = [
        list(range(0, n_traits)),
        list(range(n_traits, 2*n_traits)),
        list(range(2*n_traits, 3*n_traits))
    ]

    # Beispiel: n_traits=2
    A1 = normalize_state(0.8*np.array([1,0]) + 0.2*np.array([0,1]))
    A2 = normalize_state(0.5*np.array([1,0]) + 0.5*np.array([0,1]))
    B1 = normalize_state(0.2*np.array([1,0]) + 0.8*np.array([0,1]))
    B2 = normalize_state(0.5*np.array([1,0]) + 0.5*np.array([0,1]))
    C1 = normalize_state(0.5*np.array([1,0]) + 0.5*np.array([0,1]))
    C2 = normalize_state(0.2*np.array([0,1]) + 0.8*np.array([1,0]))
    psiA = np.kron(A1, A2)
    psiB = np.kron(B1, B2)
    psiC = np.kron(C1, C2)
    psi = np.kron(np.kron(psiA, psiB), psiC)
    psi = normalize_state(psi)

    print(f"Norm von psi = {np.linalg.norm(psi):.6f}\n")

    # -> Vor XX
    print("Vor der XX-Wechselwirkung:\nFein-granulare Messresultate:")
    S_fine_init = print_fine_grained(psi, N, threshold=threshold)
    print("\nCoarse-Grained Messresultate:")
    S_coarse_init = print_coarse_grained(psi, N, n_traits=n_traits, threshold=threshold)

    rho_init = np.outer(psi, psi.conjugate())
    vn_global_init = compute_vn_entropy(rho_init)
    print(f"Von-Neumann Entropy (global, vor XX) = {vn_global_init:.6f} bits")

    phi_init = compute_phi(rho_init, partitions, N)
    print(f"Phi (A|B|C) VOR XX-Kopplung = {phi_init:.6f} bits")

    print("\n=== Subsystem-Entropien VOR XX-Wechselwirkung ===")
    print_subsystem_entropies(rho_init, partitions[0], n_traits, label="Person A", threshold=threshold)
    print_subsystem_entropies(rho_init, partitions[1], n_traits, label="Person B", threshold=threshold)
    print_subsystem_entropies(rho_init, partitions[2], n_traits, label="Person C", threshold=threshold)

    # -> XX-Kopplung
    gamma = 0.5
    U_4 = make_xx_gate_4x4(gamma)
    pairs = []
    for k in range(n_traits):
        pairs.append((k, n_traits+k))
        pairs.append((k, 2*n_traits+k))
        pairs.append((n_traits+k, 2*n_traits+k))
    psi_xx = psi.copy()
    for (q1,q2) in pairs:
        psi_xx = apply_two_qubit_gate(psi_xx, U_4, q1,q2, N)

    print("\nNach der XX-Wechselwirkung:")
    print("Fein-granulare Messresultate:")
    S_fine_xx = print_fine_grained(psi_xx, N, threshold=threshold)
    print("\nCoarse-Grained Messresultate:")
    S_coarse_xx = print_coarse_grained(psi_xx, N, n_traits=n_traits, threshold=threshold)

    rho_xx = np.outer(psi_xx, psi_xx.conjugate())
    vn_global_xx = compute_vn_entropy(rho_xx)
    print(f"Von-Neumann Entropy (global, nach XX) = {vn_global_xx:.6f} bits")

    phi_xx = compute_phi(rho_xx, partitions, N)
    print(f"Phi (A|B|C) NACH XX-Kopplung = {phi_xx:.6f} bits")

    print("\n=== Subsystem-Entropien NACH XX-Wechselwirkung ===")
    print_subsystem_entropies(rho_xx, partitions[0], n_traits, label="Person A", threshold=threshold)
    print_subsystem_entropies(rho_xx, partitions[1], n_traits, label="Person B", threshold=threshold)
    print_subsystem_entropies(rho_xx, partitions[2], n_traits, label="Person C", threshold=threshold)

    # -> Dissipation
    rho_diss = psi_xx
    for _ in range(num_diss_steps):
        rho_diss = apply_dissipation_step(rho_diss, range(N), p_dissip, N)

    print("\nNach Dissipation Step(s) (Amplitude-Damping, Kraus-basiert):")
    print("Fein-granulare Messresultate:")
    S_fine_diss = print_fine_grained(rho_diss, N, threshold=threshold)
    print("\nCoarse-Grained Messresultate:")
    S_coarse_diss = print_coarse_grained(rho_diss, N, n_traits=n_traits, threshold=threshold)

    if rho_diss.ndim == 1:
        rho_diss = np.outer(rho_diss, rho_diss.conjugate())

    vn_global_diss = compute_vn_entropy(rho_diss)
    print(f"Von-Neumann Entropy (global, nach Dissipation) = {vn_global_diss:.6f} bits")

    phi_diss = compute_phi(rho_diss, partitions, N)
    print(f"Phi (A|B|C) NACH DISSIPATION = {phi_diss:.6f} bits")

    print("\n=== Subsystem-Entropien NACH DISSIPATION ===")
    print_subsystem_entropies(rho_diss, partitions[0], n_traits, label="Person A", threshold=threshold)
    print_subsystem_entropies(rho_diss, partitions[1], n_traits, label="Person B", threshold=threshold)
    print_subsystem_entropies(rho_diss, partitions[2], n_traits, label="Person C", threshold=threshold)

    print("\n===ENDE===\n")


if __name__=="__main__":
    run_program_with_handcrafted_traits(n_traits=2, threshold=0.01, p_dissip=0.5, num_diss_steps=5)


==== TEAM-SYSTEM mit 2 Traits/Person, p_dissip=0.5, 5 Dissipationsschritte ====

Norm von psi = 1.000000

Vor der XX-Wechselwirkung:
Fein-granulare Messresultate:
Fine-Grained Observational Entropy = 3.968271 bits

Fein-granulare Messresultate
(nur p > 0.01)
001000: 0.104213
001010: 0.104213
001100: 0.104213
001110: 0.104213
011000: 0.104213
011010: 0.104213
011100: 0.104213
011110: 0.104213

Coarse-Grained Messresultate:
Coarse-Grained Observational Entropy = 5.296159 bits

Coarse-Grained Messresultate
(nur p > 0.01)
000 (Volumen=27): p = 0.498728
001 (Volumen=9): p = 0.015113
010 (Volumen=9): p = 0.443314
011 (Volumen=3): p = 0.013434
100 (Volumen=9): p = 0.015113
110 (Volumen=3): p = 0.013434
Von-Neumann Entropy (global, vor XX) = 0.000000 bits
Phi (A|B|C) VOR XX-Kopplung = -0.000000 bits

=== Subsystem-Entropien VOR XX-Wechselwirkung ===

--- Subsystem Person A (Qubits=[0, 1]) ---
  Von-Neumann Entropy (Subsystem) = 0.000000 bits
  Fine-Entropy (Subsystem) = 1.322757 bits
  (nur p 